In [53]:
import pandas as pd
import wandb

import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

In [54]:
pd.set_option('display.max_rows', 100)

---

In [55]:
def get_runs_df(project, entity="jurujin"):
    api = wandb.Api()
    runs = api.runs(entity + "/" + project)

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        summary_list.append(run.summary._json_dict)
        config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})
        name_list.append(run.name)


    summary_df = pd.DataFrame(summary_list)
    config_df = pd.DataFrame(config_list)

    df = pd.concat([summary_df.reset_index(drop=True), config_df.reset_index(drop=True)], axis=1)
    df["name"] = name_list

    return df.sort_values(by=["w_radius", "msg_scaler"])

In [56]:
projects = [
    "msg_grid_srch",    # V
    "msg_diff_att",     # V
    "msg_all_attacks",      # V
    "fid_msg_grid_srch_gen",    # doing 1 run for r=4 s=60
    "fid_msg_grid_srch_gt",     # V
    "fid_msg_r_gen",    # V
    "fid_msg_r_gt"      # V
]

detection_projects = [
    "msg_grid_srch",    # V
    "msg_diff_att",     # V
    "msg_all_attacks"
]

fid_projects = [
    "fid_msg_grid_srch_gen",    # doing 1 run for r=4 s=60
    "fid_msg_grid_srch_gt",     # V
    "fid_msg_r_gen",
    "fid_msg_r_gt"
]

detection_cols = [
    "TPR@1%FPR", "acc", "auc",
    "w_clip_score_mean",
    "w_det_dist_mean",
    "no_w_det_dist_mean",
    "w_det_dist_std",
    "no_w_det_dist_std",
    "Bit_acc", "Word_acc",
    "msg", "w_radius", "msg_scaler",

    "jpeg_ratio", "crop_scale", "crop_ratio", "gaussian_blur_r", "gaussian_std", "brightness_factor", "r_degree"
]

fid_cols = [
    "psnr_w", "ssim_w",
    "psnr_no_w", "ssim_no_w",
    "fid_w", "fid_no_w",
    "msg", "w_radius", "msg_scaler",

]

In [57]:
for project in detection_projects:
    get_runs_df(project).to_csv(f"./detection/{project}.csv", index=False, columns=detection_cols)

for project in fid_projects:
    get_runs_df(project).to_csv(f"./fid/{project}.csv", index=False, columns=fid_cols)